In [18]:
import os
import cv2
import numpy as np
import face_recognition as fr
from datetime import datetime
import csv

In [19]:
path='attendance_students'
student_list=os.listdir(path)
student_list

['Aflah.jpg',
 'Akash Francis.jpeg',
 'Amala Raju.jpg',
 'Anakha cs.jpg',
 'Aromal pv.jpg',
 'Fizha.jpg',
 'Krishna Priya.jpeg',
 'Salman Faris.jpg',
 'Sinan Shereef.jpg',
 'Vishak s.jpg']

In [20]:
imgs=[]
student_names=[]
for i in student_list:
    img_path=os.path.join(path,i)
    img_read=cv2.imread(img_path)
    imgs.append(img_read)
    student_names.append(i.split('.')[0])

print(student_names)

['Aflah', 'Akash Francis', 'Amala Raju', 'Anakha cs', 'Aromal pv', 'Fizha', 'Krishna Priya', 'Salman Faris', 'Sinan Shereef', 'Vishak s']


In [21]:
def faceencode(images):
    encode_list=[]
    for i in imgs:
        rgb_img=cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        face_loc=fr.face_locations(rgb_img)
        face_encod=fr.face_encodings(rgb_img,face_loc)[0]
        encode_list.append(face_encod)
    return encode_list

In [22]:
encode_known_face=faceencode(imgs)
encode_known_face

[array([-0.16899319, -0.02000539,  0.06788323, -0.02252333,  0.01286584,
        -0.15478615, -0.02613035, -0.09578927,  0.08843823, -0.08312912,
         0.27158663, -0.07623373, -0.11131416, -0.19847356,  0.02824538,
         0.11489733, -0.16710055, -0.13518411, -0.04178841,  0.00367125,
         0.04375841, -0.08151795,  0.02632018,  0.10927546, -0.15237711,
        -0.43139717, -0.14752477, -0.08957843,  0.00970827, -0.05916118,
        -0.04461581,  0.04601384, -0.19394775, -0.07281251, -0.02748128,
         0.0807981 ,  0.04586955, -0.04257964,  0.20181866,  0.0027088 ,
        -0.24433373,  0.09174161,  0.05662903,  0.29889995,  0.19188309,
         0.02315924,  0.03479676,  0.00966078,  0.1223262 , -0.21165951,
         0.11461624,  0.09924453,  0.15569231, -0.01052828,  0.06604408,
        -0.04108325, -0.01406598,  0.11613292, -0.19988826,  0.10654274,
         0.042405  , -0.07752951,  0.03014861,  0.00374276,  0.18094146,
         0.16063431, -0.11991021, -0.13357495,  0.1

In [23]:
marked_students = set()  # prevents duplicate attendance
attendance_file = "attendance.csv"

In [24]:
def mark_attendance(name):
    if name not in marked_students:
        now = datetime.now()
        time_str = now.strftime("%H:%M:%S")

        with open(attendance_file, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([name, time_str])

        marked_students.add(name)


In [25]:
video=cv2.VideoCapture(0)
while True:
    sucess,img=video.read()
    img1=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    face_loc1=fr.face_locations(img1)
    face_encod1=fr.face_encodings(img1,face_loc1)
    for enc,loc in zip(face_encod1,face_loc1):
        matches=fr.compare_faces(encode_known_face,enc)
        face_dis=fr.face_distance(encode_known_face,enc)
        match_index=np.argmin(face_dis)
        names = "Unknown"
        if matches[match_index]==True:
            names = student_names[match_index].upper()
            mark_attendance(names)
        
        y1,x2,y2,x1=loc
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,0,255),2)
        cv2.putText(img,names,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
    cv2.imshow('Frame',img)
    if cv2.waitKey(1)&0XFF==ord('q'):
        break
video.release()
cv2.destroyAllWindows()